# Aging Stock

In [31]:
# --- Load Datasets ---
import pandas as pd

inventory = pd.read_csv("sources data/Inventory.csv")
inbound = pd.read_csv("sources data/Inbound.csv")
material = pd.read_csv("sources data/MaterialMaster.csv")

In [32]:
# --- Clean data ---
inbound["INBOUND_DATE"] = pd.to_datetime(inbound["INBOUND_DATE"], errors='coerce')
inventory = inventory.dropna(subset=["BATCH_NUMBER"])
inventory["BALANCE_AS_OF_DATE"] = pd.to_datetime(inventory["BALANCE_AS_OF_DATE"])
inventory["UNRESTRICTED_STOCK"] = pd.to_numeric(inventory["UNRESTRICTED_STOCK"], errors="coerce").fillna(0)

In [33]:
# --- Convert KG to MT ---
inventory.loc[inventory["STOCK_UNIT"] == "KG", "UNRESTRICTED_STOCK"] /= 1000
inventory["STOCK_UNIT"] = "MT"

# --- Convert Currency to USD ---
currency_rates = {"CNY": 0.14, "SGD": 0.74}
inventory["STOCK_SELL_VALUE"] = pd.to_numeric(inventory["STOCK_SELL_VALUE"], errors="coerce").fillna(0)
inventory["CURRENCY"] = inventory["CURRENCY"].str.upper()
inventory["SELL_VALUE_USD"] = inventory.apply(
    lambda row: row["STOCK_SELL_VALUE"] * currency_rates.get(row["CURRENCY"], 1), axis=1)

C:\Users\Windows 10\AppData\Local\Temp\ipykernel_6484\3014531453.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 0.164  0.    70.72  ...  0.325  3.     0.75 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  inventory.loc[inventory["STOCK_UNIT"] == "KG", "UNRESTRICTED_STOCK"] /= 1000


In [34]:
# --- Prepare Inbound Date (Earliest per PLANT + MATERIAL) ---
inbound["INBOUND_DATE"] = pd.to_datetime(inbound["INBOUND_DATE"])
inbound_grouped = (
    inbound.groupby(["PLANT_NAME", "MATERIAL_NAME", "INBOUND_DATE"])["NET_QUANTITY_MT"]
    .sum().reset_index()
)
earliest_inbound = (
    inbound_grouped.groupby(["PLANT_NAME", "MATERIAL_NAME"])["INBOUND_DATE"]
    .min().reset_index()
)
inventory = inventory.merge(earliest_inbound, on=["PLANT_NAME", "MATERIAL_NAME"], how="left")
inventory.rename(columns={"INBOUND_DATE": "LOT_INBOUND_DATE"}, inplace=True)

In [35]:
# --- Add Material Shelf Life Info ---
material["SHELF_LIFE_IN_MONTH"] = pd.to_numeric(material["SHELF_LIFE_IN_MONTH"], errors="coerce").fillna(0)
material["DOWNGRADE_VALUE_LOST_PERCENT"] = pd.to_numeric(material["DOWNGRADE_VALUE_LOST_PERCENT"], errors="coerce").fillna(0)
inventory = inventory.merge(material[["MATERIAL_NAME", "SHELF_LIFE_IN_MONTH", "DOWNGRADE_VALUE_LOST_PERCENT"]],
                            on="MATERIAL_NAME", how="left")

In [36]:
# --- Calculate Expiry and Days Left ---
inventory["LOT_INBOUND_DATE"] = pd.to_datetime(inventory["LOT_INBOUND_DATE"])
inventory["SHELF_LIFE_DAYS"] = inventory["SHELF_LIFE_IN_MONTH"] * 30
inventory["EXPIRY_DATE"] = inventory["LOT_INBOUND_DATE"] + pd.to_timedelta(inventory["SHELF_LIFE_DAYS"], unit="D")
inventory["DAYS_LEFT"] = (inventory["EXPIRY_DATE"] - inventory["BALANCE_AS_OF_DATE"]).dt.days

In [37]:
# --- Categorize Stock Status ---
def categorize_status(days):
    if days > 90:
        return "Healthy"
    elif 31 <= days <= 90:
        return "Near Expiry"
    elif 1 <= days <= 30:
        return "Urgent to Clear"
    else:
        return "Expired"

inventory["STATUS"] = inventory["DAYS_LEFT"].apply(categorize_status)

In [38]:
# --- Calculate Loss (Only Expired Stock) ---
inventory["LOSS_VALUE_USD"] = 0.0
expired_mask = inventory["STATUS"] == "Expired"
inventory.loc[expired_mask, "LOSS_VALUE_USD"] = (
    inventory.loc[expired_mask, "SELL_VALUE_USD"] *
    (inventory.loc[expired_mask, "DOWNGRADE_VALUE_LOST_PERCENT"] / 100))

In [39]:
# --- Final Aging Report ---
aging_report = inventory[[
    "PLANT_NAME", "MATERIAL_NAME", "BATCH_NUMBER", "BALANCE_AS_OF_DATE",
    "UNRESTRICTED_STOCK", "SELL_VALUE_USD", "STATUS", "DAYS_LEFT", "LOSS_VALUE_USD"
]].copy()
aging_report.rename(columns={"UNRESTRICTED_STOCK": "STOCK_MT"}, inplace=True)

In [40]:
# --- Summary Table ---
summary_by_status = aging_report.groupby("STATUS").agg({
    "STOCK_MT": "sum",
    "SELL_VALUE_USD": "sum",
    "LOSS_VALUE_USD": "sum"
}).reset_index()

In [41]:
# --- Export ---
aging_report.to_csv("Aging_Stock_Report.csv", index=False)
summary_by_status.to_csv("Aging_Summary_Report.csv", index=False)

# Outbond Prediction

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt

inbound_df = pd.read_csv("sources data/Inbound.csv")
inventory_df = pd.read_csv("sources data/Inventory.csv")
outbound_df = pd.read_csv("sources data/Outbound.csv")
inbound_df.columns = inbound_df.columns.str.strip()
outbound_df.columns = outbound_df.columns.str.strip()
inventory_df.columns = inventory_df.columns.str.strip()

outbound_df = outbound_df.dropna()
inventory_df = inventory_df.dropna()

In [47]:
!python -m pip install prophet
from prophet import Prophet
from dateutil.relativedelta import relativedelta

# Data Prep
outbound_df['OUTBOUND_DATE'] = pd.to_datetime(outbound_df['OUTBOUND_DATE'])
outbound_2024 = outbound_df[
    (outbound_df['OUTBOUND_DATE'].dt.year == 2024) &
    (outbound_df['OUTBOUND_DATE'].dt.month <= 11)
]

outbound_2024['YearMonth'] = outbound_2024['OUTBOUND_DATE'].dt.to_period('M').dt.to_timestamp()

grouped = outbound_2024.groupby(['PLANT_NAME', 'MATERIAL_NAME', 'YearMonth'])['NET_QUANTITY_MT'].sum().reset_index()

# Model
def forecast_december(df_group):
    df_prophet = df_group.rename(columns={'YearMonth': 'ds', 'NET_QUANTITY_MT': 'y'})[['ds', 'y']]
    if len(df_prophet) < 3:
        return None

    model = Prophet()
    model.fit(df_prophet)

    last_date = df_prophet['ds'].max()
    target_date = pd.to_datetime('2024-12-01')
    months_to_forecast = (target_date.year - last_date.year) * 12 + (target_date.month - last_date.month)

    if months_to_forecast <= 0:
        return None

    future = model.make_future_dataframe(periods=months_to_forecast, freq='MS')
    forecast = model.predict(future)

    december_row = forecast[forecast['ds'] == target_date]
    if december_row.empty:
        return None

    return december_row.iloc[0]['yhat']

# Next month forecast
forecast_results = []
for (plant, material), group in grouped.groupby(['PLANT_NAME', 'MATERIAL_NAME']):
    forecast_value = forecast_december(group)
    if forecast_value is not None:
        forecast_value = max(forecast_value, 0)
        forecast_results.append({
            'PLANT_NAME': plant,
            'MATERIAL_NAME': material,
            'Raw_Forecast_Quantity': forecast_value
        })

forecast_df = pd.DataFrame(forecast_results)

plant_targets = {
    'SINGAPORE-WAREHOUSE': 13.97,
    'CHINA-WAREHOUSE': 21.80
}

final_results = []

# Normalize
for plant, group in forecast_df.groupby('PLANT_NAME'):
    if plant in plant_targets:
        total_raw = group['Raw_Forecast_Quantity'].sum()
        target_kt = plant_targets[plant]
        scaling_factor = (target_kt * 1000) / total_raw

        group = group.copy()
        group['Adjusted_Forecast_MT'] = group['Raw_Forecast_Quantity'] * scaling_factor
        group['Adjusted_Forecast_KT'] = (group['Adjusted_Forecast_MT'] / 1000).round(2)
        final_results.append(group)

outbound_predicted_df = pd.concat(final_results, ignore_index=True)

# Summary
summary_by_plant = outbound_predicted_df.groupby('PLANT_NAME')['Adjusted_Forecast_KT'].sum().reset_index()
summary_by_plant['Adjusted_Forecast_KT'] = summary_by_plant['Adjusted_Forecast_KT'].round(2).astype(str) + ' KT'

print("\n Summary Total Space Use")
print(summary_by_plant)

   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.3 MB 4.8 MB/s eta 0:00:03
   ------- -------------------------------- 2.4/13.3 MB 5.4 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/13.3 MB 5.6 MB/s eta 0:00:02
   ----------- ---------------------------- 3.9/13.3 MB 4.8 MB/s eta 0:00:02
   ------------- -------------------------- 4.5/13.3 MB 4.3 MB/s eta 0:00:03
   ---------------- ----------------------- 5.5/13.3 MB 4.3 MB/s eta 0:00:02
   ------------------- -------------------- 6.6/13.3 MB 4.4 MB/s eta 0:00:02
   ---------------------- ----------------- 7.3/13.3 MB 4.3 MB/s eta 0:00:02
   ------------------------ --------------- 8.1/13.3 MB 4.4 MB/s eta 0:00:02
   --------------------------- ------------ 9.2/13.3 MB 4.4 MB/s eta 0:00:01
   ------------------------------ --------- 10.2/13.3 MB 4.3 MB/s eta 0:00:01
   --------------------------------- ------ 11.0/13.3 MB 4.3 MB/s eta 0:00:01
   -

C:\Users\Windows 10\AppData\Local\Temp\ipykernel_6484\697974626.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outbound_2024['YearMonth'] = outbound_2024['OUTBOUND_DATE'].dt.to_period('M').dt.to_timestamp()
13:25:27 - cmdstanpy - INFO - Chain [1] start processing
13:25:29 - cmdstanpy - INFO - Chain [1] done processing
13:25:29 - cmdstanpy - INFO - Chain [1] start processing
13:25:29 - cmdstanpy - INFO - Chain [1] done processing
13:25:29 - cmdstanpy - INFO - Chain [1] start processing
13:25:29 - cmdstanpy - INFO - Chain [1] done processing
13:25:29 - cmdstanpy - INFO - Chain [1] start processing
13:25:30 - cmdstanpy - INFO - Chain [1] done processing
13:25:30 - cmdstanpy - INFO - Chain [1] start processing
13:25:30 - cmdstanpy - INFO - Chai


 Summary Total Space Use
            PLANT_NAME Adjusted_Forecast_KT
0      CHINA-WAREHOUSE              21.8 KT
1  SINGAPORE-WAREHOUSE             13.95 KT


In [48]:
outbound_predicted_df

,PLANT_NAME,MATERIAL_NAME,Raw_Forecast_Quantity,Adjusted_Forecast_MT,Adjusted_Forecast_KT
0,CHINA-WAREHOUSE,MAT-0001,613.429344,489.748925,0.49
1,CHINA-WAREHOUSE,MAT-0002,216.235670,172.637954,0.17
2,CHINA-WAREHOUSE,MAT-0004,0.000000,0.000000,0.00
3,CHINA-WAREHOUSE,MAT-0007,372.964364,297.766806,0.30
4,CHINA-WAREHOUSE,MAT-0009,0.000000,0.000000,0.00
...,...,...,...,...,...
295,SINGAPORE-WAREHOUSE,MAT-0415,149.385413,114.678758,0.11
296,SINGAPORE-WAREHOUSE,MAT-0416,0.000000,0.000000,0.00
297,SINGAPORE-WAREHOUSE,MAT-0421,0.000000,0.000000,0.00
298,SINGAPORE-WAREHOUSE,MAT-0423,75.599013,58.035124,0.06


# Overflow Model Prediction

## Overflow Material Calculation

In [49]:
# Data Prep
outbound_df["OUTBOUND_DATE"] = pd.to_datetime(outbound_df["OUTBOUND_DATE"])
outbound_df["Month"] = outbound_df["OUTBOUND_DATE"].dt.to_period("M")

outbound_2024 = outbound_df[
    (outbound_df["Month"].dt.year == 2024) &
    (outbound_df["Month"].dt.month <= 11) &
    (outbound_df["NET_QUANTITY_MT"] > 0)
]

monthly_sales = (
    outbound_2024
    .groupby(["PLANT_NAME", "MATERIAL_NAME", "Month"])
    .agg({"NET_QUANTITY_MT": "sum"})
    .reset_index()
)

active_months_df = (
    monthly_sales
    .groupby(["PLANT_NAME", "MATERIAL_NAME"])["Month"]
    .nunique()
    .reset_index()
    .rename(columns={"Month": "ACTIVE_MONTHS"})
)

out_mt_df = (
    monthly_sales
    .groupby(["PLANT_NAME", "MATERIAL_NAME"])["NET_QUANTITY_MT"]
    .sum()
    .reset_index()
    .rename(columns={"NET_QUANTITY_MT": "OUT_MT"})
)

final_df = pd.merge(out_mt_df, active_months_df, on=["PLANT_NAME", "MATERIAL_NAME"])

max_capacity_dict = {
    "CHINA-WAREHOUSE": 70,
    "SINGAPORE-WAREHOUSE": 53.5
}
final_df["Max_Capacity"] = final_df["PLANT_NAME"].map(max_capacity_dict)
final_df["Max_Capacity"] = final_df["Max_Capacity"] * 1000  # MT → KG

final_df["OUT_RATE_%"] = (final_df["OUT_MT"] / final_df["Max_Capacity"]) * 100

inbound_df["INBOUND_DATE"] = pd.to_datetime(inbound_df["INBOUND_DATE"])
inbound_df["Month"] = inbound_df["INBOUND_DATE"].dt.to_period("M")

inbound_2024 = inbound_df[
    (inbound_df["Month"].dt.year == 2024) &
    (inbound_df["Month"].dt.month <= 11) &
    (inbound_df["NET_QUANTITY_MT"] > 0)
]

in_mt_df = (
    inbound_2024
    .groupby(["PLANT_NAME", "MATERIAL_NAME"])["NET_QUANTITY_MT"]
    .sum()
    .reset_index()
    .rename(columns={"NET_QUANTITY_MT": "IN_MT"})
)

in_mt_df["IN_MT_PER_MONTH"] = in_mt_df["IN_MT"] / 11

final_df = pd.merge(final_df, in_mt_df, on=["PLANT_NAME", "MATERIAL_NAME"], how="left")
final_df["IN_MT"] = final_df["IN_MT"].fillna(0)
final_df["IN_MT_PER_MONTH"] = final_df["IN_MT_PER_MONTH"].fillna(0)

inventory_df = inventory_df[~inventory_df["BATCH_NUMBER"].str.upper().str.contains("SCRAP", na=False)]
inventory_df["BATCH_NUMBER"] = inventory_df["BATCH_NUMBER"].astype(str)

inventory_df["BALANCE_AS_OF_DATE"] = pd.to_datetime(inventory_df["BALANCE_AS_OF_DATE"])
inventory_df["Month"] = inventory_df["BALANCE_AS_OF_DATE"].dt.to_period("M")

last_inventory_date = inventory_df[
    inventory_df["Month"] == "2024-11"
]["BALANCE_AS_OF_DATE"].max()

latest_inventory_df = inventory_df[
    inventory_df["BALANCE_AS_OF_DATE"] == last_inventory_date
]

stock_df = (
    latest_inventory_df
    .groupby(["PLANT_NAME", "MATERIAL_NAME"])
    .agg({
        "UNRESTRICTED_STOCK": "sum",
        "BATCH_NUMBER": lambda x: list(x.unique())
    })
    .reset_index()
)

# Merge
final_df = pd.merge(final_df, stock_df, on=["PLANT_NAME", "MATERIAL_NAME"], how="left")
final_df["UNRESTRICTED_STOCK"] = final_df["UNRESTRICTED_STOCK"] / 1000
final_df["UNRESTRICTED_STOCK"] = final_df["UNRESTRICTED_STOCK"].fillna(0)
final_df["BATCH_NUMBER"] = final_df["BATCH_NUMBER"].apply(lambda x: x if isinstance(x, list) else [])

final_columns = [
    "PLANT_NAME", "MATERIAL_NAME", "BATCH_NUMBER", "Max_Capacity",
    "OUT_MT", "OUT_RATE_%",
    "IN_MT", "IN_MT_PER_MONTH",
    "UNRESTRICTED_STOCK", "PREDCITED_OUTBOUND_NEXT_MONTH", "PREDCITED_TOTAL_OUTBOUND_NEXT_MONTH"
]

plant_targets = {
    'SINGAPORE-WAREHOUSE': 13.97 * 1000,
    'CHINA-WAREHOUSE': 21.80 * 1000
}

final_df = final_df.merge(
    outbound_predicted_df[['PLANT_NAME', 'MATERIAL_NAME', 'Adjusted_Forecast_MT']],
    on=['PLANT_NAME', 'MATERIAL_NAME'],
    how='left'
)

final_df['Adjusted_Forecast_MT'] = final_df['Adjusted_Forecast_MT'].fillna(0)

final_df = final_df.rename(columns={'Adjusted_Forecast_MT': 'PREDCITED_OUTBOUND_NEXT_MONTH'})

final_df['PREDCITED_TOTAL_OUTBOUND_NEXT_MONTH'] = final_df['PLANT_NAME'].map(plant_targets)

final_df['PREDCITED_TOTAL_OUTBOUND_NEXT_MONTH'] = final_df['PREDCITED_TOTAL_OUTBOUND_NEXT_MONTH'].fillna(0)

final_df = final_df[final_columns]

final_df

,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,Max_Capacity,OUT_MT,OUT_RATE_%,IN_MT,IN_MT_PER_MONTH,UNRESTRICTED_STOCK,PREDCITED_OUTBOUND_NEXT_MONTH,PREDCITED_TOTAL_OUTBOUND_NEXT_MONTH
0,CHINA-WAREHOUSE,MAT-0001,"[V6661A, V1821A, V1214A, V8768A, V4876A, V5763...",70000.0,8117.040,11.595771,7981.500,725.590909,520.325,489.748925,21800.0
1,CHINA-WAREHOUSE,MAT-0002,"[N2666A, N5378A]",70000.0,414.050,0.591500,420.750,38.250000,7.875,172.637954,21800.0
2,CHINA-WAREHOUSE,MAT-0003,"[8045, 3882, 2397, 5767]",70000.0,73.370,0.104814,78.336,7.121455,117.504,0.000000,21800.0
3,CHINA-WAREHOUSE,MAT-0004,[],70000.0,5.930,0.008471,0.000,0.000000,0.000,0.000000,21800.0
4,CHINA-WAREHOUSE,MAT-0005,[],70000.0,0.030,0.000043,0.000,0.000000,0.000,0.000000,21800.0
...,...,...,...,...,...,...,...,...,...,...,...
457,SINGAPORE-WAREHOUSE,MAT-0423,"[M9457B, M2655A, M7790A, M9321A]",53500.0,294.460,0.550393,371.250,33.750000,98.900,58.035124,13970.0
458,SINGAPORE-WAREHOUSE,MAT-0424,"[M3515A, M4013A]",53500.0,235.125,0.439486,1089.000,99.000000,828.750,0.000000,13970.0
459,SINGAPORE-WAREHOUSE,MAT-0426,"[P3753A, P1159A, P1821A]",53500.0,25.500,0.047664,0.000,0.000000,3901.500,0.000000,13970.0
460,SINGAPORE-WAREHOUSE,MAT-0427,[],53500.0,15.000,0.028037,0.000,0.000000,0.000,0.000000,13970.0


In [50]:
# Data Calculation
final_df["NEXT_MONTH_STOCK"] = final_df["UNRESTRICTED_STOCK"] + final_df["IN_MT_PER_MONTH"]

plant_sum = final_df.groupby("PLANT_NAME").agg({
    "NEXT_MONTH_STOCK": "sum",
    "Max_Capacity": "first"
}).reset_index()

plant_sum["OVERFLOW_AMOUNT"] = plant_sum["NEXT_MONTH_STOCK"] - plant_sum["Max_Capacity"]
plant_sum["WAREHOUSE_OVERFLOW"] = plant_sum["NEXT_MONTH_STOCK"] > plant_sum["Max_Capacity"]

final_df["OVER_SUPPLY"] = final_df["IN_MT_PER_MONTH"] - final_df["PREDCITED_OUTBOUND_NEXT_MONTH"]
final_df["IS_OVER_SUPPLY"] = final_df["OVER_SUPPLY"] > 0

final_df = pd.merge(
    final_df,
    plant_sum[["PLANT_NAME", "WAREHOUSE_OVERFLOW"]],
    on="PLANT_NAME",
    how="left"
)

final_df["DESERVED_SPACE_RATIO"] = final_df["PREDCITED_OUTBOUND_NEXT_MONTH"] / final_df["PREDCITED_TOTAL_OUTBOUND_NEXT_MONTH"]
final_df["DESERVED_STOCK_MT"] = final_df["DESERVED_SPACE_RATIO"] * final_df["Max_Capacity"]

final_df["OVER_DESERVED"] = final_df["NEXT_MONTH_STOCK"] - final_df["DESERVED_STOCK_MT"]

final_df["RISK_OVERFLOW"] = (
    (final_df["WAREHOUSE_OVERFLOW"] == True) &
    (final_df["OVER_DESERVED"] > 0)
)

final_df

,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,Max_Capacity,OUT_MT,OUT_RATE_%,IN_MT,IN_MT_PER_MONTH,UNRESTRICTED_STOCK,PREDCITED_OUTBOUND_NEXT_MONTH,PREDCITED_TOTAL_OUTBOUND_NEXT_MONTH,NEXT_MONTH_STOCK,OVER_SUPPLY,IS_OVER_SUPPLY,WAREHOUSE_OVERFLOW,DESERVED_SPACE_RATIO,DESERVED_STOCK_MT,OVER_DESERVED,RISK_OVERFLOW
0,CHINA-WAREHOUSE,MAT-0001,"[V6661A, V1821A, V1214A, V8768A, V4876A, V5763...",70000.0,8117.040,11.595771,7981.500,725.590909,520.325,489.748925,21800.0,1245.915909,235.841984,True,True,0.022466,1572.588291,-326.672382,False
1,CHINA-WAREHOUSE,MAT-0002,"[N2666A, N5378A]",70000.0,414.050,0.591500,420.750,38.250000,7.875,172.637954,21800.0,46.125000,-134.387954,False,True,0.007919,554.342056,-508.217056,False
2,CHINA-WAREHOUSE,MAT-0003,"[8045, 3882, 2397, 5767]",70000.0,73.370,0.104814,78.336,7.121455,117.504,0.000000,21800.0,124.625455,7.121455,True,True,0.000000,0.000000,124.625455,True
3,CHINA-WAREHOUSE,MAT-0004,[],70000.0,5.930,0.008471,0.000,0.000000,0.000,0.000000,21800.0,0.000000,0.000000,False,True,0.000000,0.000000,0.000000,False
4,CHINA-WAREHOUSE,MAT-0005,[],70000.0,0.030,0.000043,0.000,0.000000,0.000,0.000000,21800.0,0.000000,0.000000,False,True,0.000000,0.000000,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,SINGAPORE-WAREHOUSE,MAT-0423,"[M9457B, M2655A, M7790A, M9321A]",53500.0,294.460,0.550393,371.250,33.750000,98.900,58.035124,13970.0,132.650000,-24.285124,False,False,0.004154,222.253338,-89.603338,False
458,SINGAPORE-WAREHOUSE,MAT-0424,"[M3515A, M4013A]",53500.0,235.125,0.439486,1089.000,99.000000,828.750,0.000000,13970.0,927.750000,99.000000,True,False,0.000000,0.000000,927.750000,False
459,SINGAPORE-WAREHOUSE,MAT-0426,"[P3753A, P1159A, P1821A]",53500.0,25.500,0.047664,0.000,0.000000,3901.500,0.000000,13970.0,3901.500000,0.000000,False,False,0.000000,0.000000,3901.500000,False
460,SINGAPORE-WAREHOUSE,MAT-0427,[],53500.0,15.000,0.028037,0.000,0.000000,0.000,0.000000,13970.0,0.000000,0.000000,False,False,0.000000,0.000000,0.000000,False


## Summary of Warehouse Space Over 80%

In [51]:
# Over 80% Summary
plant_capacity_check = final_df.groupby("PLANT_NAME").agg({
    "NEXT_MONTH_STOCK": "sum",
    "Max_Capacity": "first"
}).reset_index()

plant_capacity_check["CUTOFF_80_PERCENT"] = plant_capacity_check["Max_Capacity"] * 0.80

plant_capacity_check["OVER_80_AMOUNT_MT"] = plant_capacity_check["NEXT_MONTH_STOCK"] - plant_capacity_check["CUTOFF_80_PERCENT"]
plant_capacity_check["OVER_80_AMOUNT_MT"] = plant_capacity_check["OVER_80_AMOUNT_MT"].clip(lower=0)

plant_capacity_check["OVER_80_AMOUNT_KT"] = (plant_capacity_check["OVER_80_AMOUNT_MT"] / 1000).round(2)

plant_capacity_check["OVER_80_PERCENT"] = ((plant_capacity_check["OVER_80_AMOUNT_MT"] / plant_capacity_check["Max_Capacity"]) * 100).round(2)

plant_capacity_check["IS_OVER_80"] = plant_capacity_check["NEXT_MONTH_STOCK"] > plant_capacity_check["CUTOFF_80_PERCENT"]

overflow_80_summary = plant_capacity_check[[
    "PLANT_NAME",
    "NEXT_MONTH_STOCK",
    "Max_Capacity",
    "CUTOFF_80_PERCENT",
    "OVER_80_AMOUNT_MT",
    "OVER_80_AMOUNT_KT",
    "OVER_80_PERCENT",
    "IS_OVER_80"
]]

print("Summary of Warehouse Space Over 80%")
overflow_80_summary

Summary of Warehouse Space Over 80%


,PLANT_NAME,NEXT_MONTH_STOCK,Max_Capacity,CUTOFF_80_PERCENT,OVER_80_AMOUNT_MT,OVER_80_AMOUNT_KT,OVER_80_PERCENT,IS_OVER_80
0,CHINA-WAREHOUSE,75788.877091,70000.0,56000.0,19788.877091,19.79,28.27,True
1,SINGAPORE-WAREHOUSE,48194.899636,53500.0,42800.0,5394.899636,5.39,10.08,True


In [54]:
# Export data
outbound_predicted_df.to_csv("SKU Outbound Prediction.csv", index=False)
final_df.to_csv("SKU Overflow Alert.csv", index=False)
overflow_80_summary.to_csv("Overflow 80 Percent Summary.csv", index=False)


# Transfer Plan

In [55]:
import pandas as pd

def calc_avg_demand_3m(outbound_df, selected_month):
    outbound = outbound_df.copy()
    outbound['OUTBOUND_DATE'] = pd.to_datetime(outbound['OUTBOUND_DATE'])
    outbound['MONTH'] = pd.to_datetime(outbound['OUTBOUND_DATE'].dt.to_period('M').astype(str))
    selected_dt = pd.to_datetime(selected_month + '-01')
    recent_3m = outbound[outbound['MONTH'] >= (selected_dt - pd.DateOffset(months=2))]
    avg_demand = recent_3m.groupby(['PLANT_NAME', 'MATERIAL_NAME'])['NET_QUANTITY_MT'].sum().reset_index()
    avg_demand['AVG_DEMAND_3M'] = avg_demand['NET_QUANTITY_MT'] / 3
    return avg_demand[['PLANT_NAME', 'MATERIAL_NAME', 'AVG_DEMAND_3M']]

def stock_transfer_recommendation_with_usd(
    df_excess, df_stock, df_capacity, outbound,
    exchange_rate,
    rent_cost_table,
    transfer_cost_per_container=75,
    container_size=24.75,
    selected_month='2024-06',
    threshold=0.8
):
    avg_demand = calc_avg_demand_3m(outbound, selected_month)

    df = df_excess.merge(avg_demand, on=['PLANT_NAME', 'MATERIAL_NAME'], how='left')
    df['AVG_DEMAND_3M'] = df['AVG_DEMAND_3M'].fillna(0)
    df = df.sort_values(by='AVG_DEMAND_3M', ascending=False).reset_index(drop=True)

    df_capacity_simple = df_capacity[['PLANT_NAME', 'CAPACITY']]
    df_stock_simple = df_stock[['PLANT_NAME', 'STOCK_SUM']]
    df_room = df_capacity_simple.merge(df_stock_simple, on='PLANT_NAME', how='left')
    df_room['STOCK_SUM'] = df_room['STOCK_SUM'].fillna(0)

    df['TARGET_PLANT'] = df['PLANT_NAME'].apply(
        lambda x: 'SINGAPORE-WAREHOUSE' if x == 'CHINA-WAREHOUSE' else 'CHINA-WAREHOUSE'
    )

    rent_usd = {
        plant: rate_per_day * exchange_rate[currency]
        for plant, (rate_per_day, currency) in rent_cost_table.items()
    }

    transfer_cost_per_mt = transfer_cost_per_container / container_size

    actions, qty_transferred, rent_qty_list, cost_transfers, cost_rents = [], [], [], [], []

    stop_process = False

    for idx, row in df.iterrows():
        if stop_process:
            actions.append('NO ACTION')
            qty_transferred.append(0)
            rent_qty_list.append(0)
            cost_transfers.append(0)
            cost_rents.append(0)
            continue

        excess_qty = row['EXCESS_QTY']
        source = row['PLANT_NAME']
        target = row['TARGET_PLANT']
        demand_target = row['AVG_DEMAND_3M']

        target_stock = df_room.loc[df_room['PLANT_NAME'] == target, 'STOCK_SUM'].values[0]
        target_capacity = df_room.loc[df_room['PLANT_NAME'] == target, 'CAPACITY'].values[0]
        max_allowed = target_capacity * threshold

        source_stock = df_room.loc[df_room['PLANT_NAME'] == source, 'STOCK_SUM'].values[0]
        source_capacity = df_room.loc[df_room['PLANT_NAME'] == source, 'CAPACITY'].values[0]

        available_room = max(0, max_allowed - target_stock)

        if demand_target == 0 or available_room == 0:
            # RENT
            needed_to_reduce = max(0, source_stock - source_capacity * threshold)
            rent_qty = min(excess_qty, needed_to_reduce)

            action = 'RENT'
            qty = 0
            cost_transfer = 0
            cost_rent = rent_qty * rent_usd[source]
            df_room.loc[df_room['PLANT_NAME'] == source, 'STOCK_SUM'] -= rent_qty

        else:
            transfer_qty = min(excess_qty, available_room)
            rent_qty = excess_qty - transfer_qty

            cost_transfer = transfer_qty * transfer_cost_per_mt
            cost_rent = rent_qty * rent_usd[source]
            cost_rent_full = excess_qty * rent_usd[source]

            if cost_transfer + cost_rent < cost_rent_full:
                # TRANSFER
                action = 'TRANSFER'
                qty = transfer_qty
                df_room.loc[df_room['PLANT_NAME'] == target, 'STOCK_SUM'] += transfer_qty
                df_room.loc[df_room['PLANT_NAME'] == source, 'STOCK_SUM'] -= excess_qty
            else:
                # RENT
                needed_to_reduce = max(0, source_stock - source_capacity * threshold)
                rent_qty = min(excess_qty, needed_to_reduce)

                action = 'RENT'
                qty = 0
                cost_transfer = 0
                cost_rent = rent_qty * rent_usd[source]
                df_room.loc[df_room['PLANT_NAME'] == source, 'STOCK_SUM'] -= rent_qty

        actions.append(action)
        qty_transferred.append(qty)
        rent_qty_list.append(rent_qty)
        cost_transfers.append(cost_transfer)
        cost_rents.append(cost_rent)

        source_stock = df_room.loc[df_room['PLANT_NAME'] == source, 'STOCK_SUM'].values[0]
        if source_stock <= source_capacity * threshold:
            stop_process = True

    df['ACTION'] = actions
    df['QTY_TRANSFERRED'] = qty_transferred
    df['EXCESS_QTY_RENTED'] = rent_qty_list
    df['COST_TRANSFER_USD'] = cost_transfers
    df['COST_RENT_USD'] = cost_rents
    df['ROOM_AFTER'] = df['TARGET_PLANT'].map(
        df_room.set_index('PLANT_NAME')['CAPACITY'] * threshold -
        df_room.set_index('PLANT_NAME')['STOCK_SUM']
    ).clip(lower=0)

    return df[[
        'MATERIAL_NAME', 'PLANT_NAME', 'EXCESS_QTY', 'TARGET_PLANT',
        'ROOM_AFTER', 'AVG_DEMAND_3M',
        'QTY_TRANSFERRED', 'EXCESS_QTY_RENTED',
        'COST_TRANSFER_USD', 'COST_RENT_USD',
        'ACTION'
    ]]

In [56]:
df_excess = pd.read_csv('SKU Overflow Alert.csv')
df_excess = df_excess.rename(columns={'EXCESS_QTY': 'OVER_DESERVED'})

# กรองเฉพาะแถวที่ RISK_OVERFLOW เป็น True
df_excess = df_excess[df_excess['RISK_OVERFLOW'] == True]

df_excess = df_excess.rename(columns={'OVER_DESERVED': 'EXCESS_QTY'})
df_stock = pd.read_csv('Overflow 80 Percent Summary.csv')
df_stock = df_stock.rename(columns=lambda x: x.strip())  # ตัด space
df_stock = df_stock.rename(columns={'NEXT_MONTH_STOCK': 'STOCK_SUM'})
df_capacity = df_stock[['PLANT_NAME', 'Max_Capacity']].rename(columns={'Max_Capacity': 'CAPACITY'})
df_stock_only = df_stock[['PLANT_NAME', 'STOCK_SUM']]

df_room = df_capacity.merge(df_stock_only, on='PLANT_NAME', how='left')
df_room['STOCK_SUM'] = df_room['STOCK_SUM'].fillna(0)

outbound = pd.read_csv('sources data/Outbound.csv', parse_dates=['OUTBOUND_DATE'], dayfirst=True)

# --- Exchange rate และ rent ---
exchange_rate = {'SGD': 0.74, 'CNY': 0.14, 'USD': 1.0}
rent_cost_table = {
    'SINGAPORE-WAREHOUSE': (15, 'SGD'),
    'CHINA-WAREHOUSE': (1.7, 'CNY')
}

result = stock_transfer_recommendation_with_usd(
    df_excess, df_stock, df_capacity, outbound,
    exchange_rate, rent_cost_table,
    selected_month='2025-01'
)

print(result)

result.to_csv('stock_transfer_recommendation_output.csv', index=False)

    MATERIAL_NAME       PLANT_NAME   EXCESS_QTY         TARGET_PLANT  \
0        MAT-0145  CHINA-WAREHOUSE   907.175798  SINGAPORE-WAREHOUSE   
1        MAT-0144  CHINA-WAREHOUSE   298.754724  SINGAPORE-WAREHOUSE   
2        MAT-0082  CHINA-WAREHOUSE   961.913568  SINGAPORE-WAREHOUSE   
3        MAT-0354  CHINA-WAREHOUSE   234.862260  SINGAPORE-WAREHOUSE   
4        MAT-0319  CHINA-WAREHOUSE   612.037378  SINGAPORE-WAREHOUSE   
..            ...              ...          ...                  ...   
104      MAT-0330  CHINA-WAREHOUSE    24.275000  SINGAPORE-WAREHOUSE   
105      MAT-0352  CHINA-WAREHOUSE    10.035152  SINGAPORE-WAREHOUSE   
106      MAT-0332  CHINA-WAREHOUSE  9295.359091  SINGAPORE-WAREHOUSE   
107      MAT-0368  CHINA-WAREHOUSE    10.750000  SINGAPORE-WAREHOUSE   
108      MAT-0392  CHINA-WAREHOUSE    28.525000  SINGAPORE-WAREHOUSE   

     ROOM_AFTER  AVG_DEMAND_3M  QTY_TRANSFERRED  EXCESS_QTY_RENTED  \
0           0.0     781.870000                0         907.17579

# Scenerio

In [57]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

inbound = pd.read_csv('sources data/Inbound.csv', parse_dates=['INBOUND_DATE'])
outbound = pd.read_csv('sources data/Outbound.csv', parse_dates=['OUTBOUND_DATE'])
stock = pd.read_csv('sources data/Inventory.csv', parse_dates=['BALANCE_AS_OF_DATE'])

if 'UNRESRICTED_STOCK' in stock.columns:
    stocky = stock.rename(columns={'UNRESRICTED_STOCK': 'UNRESTRICTED_STOCK'})

inbound = inbound.rename(columns={'INBOUND_DATE': 'DATE', 'NET_QUANTITY_MT': 'INBOUND'})
outbound = outbound.rename(columns={'OUTBOUND_DATE': 'DATE', 'NET_QUANTITY_MT': 'OUTBOUND'})
stock = stock.rename(columns={'BALANCE_AS_OF_DATE': 'DATE'})

inbound['DATE'] = pd.to_datetime(inbound['DATE'], dayfirst=True, errors='coerce')
outbound['DATE'] = pd.to_datetime(outbound['DATE'], dayfirst=True, errors='coerce')
stock['DATE'] = pd.to_datetime(stock['DATE'], dayfirst=True, errors='coerce')

inbound['MONTH'] = inbound['DATE'].dt.to_period('M').dt.to_timestamp()
inbound_monthly = inbound.groupby(['PLANT_NAME', 'MATERIAL_NAME', 'MONTH'])['INBOUND'].sum().reset_index()

outbound['MONTH'] = outbound['DATE'].dt.to_period('M').dt.to_timestamp()
outbound_monthly = outbound.groupby(['PLANT_NAME', 'MATERIAL_NAME', 'MONTH'])['OUTBOUND'].sum().reset_index()

stock['MONTH'] = stock['DATE'].dt.to_period('M').dt.to_timestamp()
stock['STOCK'] = stock['UNRESTRICTED_STOCK'] / 1000

df = pd.merge(stock[['PLANT_NAME', 'MATERIAL_NAME', 'MONTH', 'STOCK']],
              inbound_monthly, on=['PLANT_NAME', 'MATERIAL_NAME', 'MONTH'], how='left')
df = pd.merge(df, outbound_monthly, on=['PLANT_NAME', 'MATERIAL_NAME', 'MONTH'], how='left')

df['INBOUND'] = df['INBOUND'].fillna(0)
df['OUTBOUND'] = df['OUTBOUND'].fillna(0)

df_combined = df.groupby(['PLANT_NAME', 'MONTH'])[['INBOUND', 'OUTBOUND', 'STOCK']].sum().reset_index()

# เตรียมฟีเจอร์
def prepare_features(df):
    df = df.copy()
    df = df.sort_values(['PLANT_NAME', 'MONTH']).reset_index(drop=True)

    for lag in range(1, 4):
        df[f'STOCK_LAG_{lag}'] = df.groupby('PLANT_NAME')['STOCK'].shift(lag)
        df[f'INBOUND_LAG_{lag}'] = df.groupby('PLANT_NAME')['INBOUND'].shift(lag)
        df[f'OUTBOUND_LAG_{lag}'] = df.groupby('PLANT_NAME')['OUTBOUND'].shift(lag)

    lag_cols = [f'{col}_LAG_{i}' for col in ['STOCK', 'INBOUND', 'OUTBOUND'] for i in range(1, 4)]
    df[lag_cols] = df[lag_cols].fillna(0)

    df['STOCK_ROLL_MEAN_3'] = df.groupby('PLANT_NAME')['STOCK'].transform(lambda x: x.rolling(3, min_periods=1).mean())
    df['INBOUND_ROLL_MEAN_3'] = df.groupby('PLANT_NAME')['INBOUND'].transform(lambda x: x.rolling(3, min_periods=1).mean())
    df['OUTBOUND_ROLL_MEAN_3'] = df.groupby('PLANT_NAME')['OUTBOUND'].transform(lambda x: x.rolling(3, min_periods=1).mean())

    df['MONTH_NUM'] = df['MONTH'].dt.month
    return df

# training data
df_feat = prepare_features(df_combined)
df_feat['TARGET'] = df_feat.groupby('PLANT_NAME')['STOCK'].shift(-1)
df_train = df_feat.dropna(subset=['TARGET'])

feature_cols = [
    'STOCK_LAG_1', 'STOCK_LAG_2', 'STOCK_LAG_3',
    'INBOUND_LAG_1', 'INBOUND_LAG_2', 'INBOUND_LAG_3',
    'OUTBOUND_LAG_1', 'OUTBOUND_LAG_2', 'OUTBOUND_LAG_3',
    'STOCK_ROLL_MEAN_3', 'INBOUND_ROLL_MEAN_3', 'OUTBOUND_ROLL_MEAN_3',
    'MONTH_NUM'
]

X_train = df_train[feature_cols]
y_train = df_train['TARGET']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

def run_scenario_combined(df, model, feature_cols, months_ahead=12,
                          inbound_factor=1.0, outbound_factor=1.0,
                          inbound_delay=0, outbound_delay=0,
                          shocks=None):
    df_ml = prepare_features(df)
    df_ml = df_ml.sort_values(['PLANT_NAME', 'MONTH']).reset_index(drop=True)

    plants = df_ml['PLANT_NAME'].unique()
    forecast_results = []

    def get_delayed_value(lag_list, delay):
        return lag_list[delay] if delay < len(lag_list) else lag_list[-1]

    for plant in plants:
        plant_df = df_ml[df_ml['PLANT_NAME'] == plant]
        monthly_df = plant_df[['MONTH', 'INBOUND', 'OUTBOUND', 'STOCK']].copy()
        monthly_df = monthly_df.groupby('MONTH').mean().reset_index().sort_values('MONTH')

        if len(monthly_df) == 0:
            continue

        last_month = monthly_df['MONTH'].max()

        lag_inbound = list(monthly_df['INBOUND'].tail(3).values[::-1])
        lag_outbound = list(monthly_df['OUTBOUND'].tail(3).values[::-1])
        lag_stock = list(monthly_df['STOCK'].tail(3).values[::-1])

        while len(lag_inbound) < 3: lag_inbound.append(0)
        while len(lag_outbound) < 3: lag_outbound.append(0)
        while len(lag_stock) < 3: lag_stock.append(0)

        for i in range(1, months_ahead + 1):
            pred_month = last_month + pd.DateOffset(months=i)
            month_num = pred_month.month

            shock_inbound_factor = inbound_factor
            shock_outbound_factor = outbound_factor

            if shocks:
                for shock in shocks:
                    if shock.get('start_month') <= month_num <= shock.get('end_month'):
                        shock_inbound_factor = shock.get('inbound_factor', shock_inbound_factor)
                        shock_outbound_factor = shock.get('outbound_factor', shock_outbound_factor)

            next_inbound = get_delayed_value(lag_inbound, inbound_delay) * shock_inbound_factor
            next_outbound = get_delayed_value(lag_outbound, outbound_delay) * shock_outbound_factor

            lag_inbound = [next_inbound] + lag_inbound[:-1]
            lag_outbound = [next_outbound] + lag_outbound[:-1]

            roll_inbound = np.mean(lag_inbound)
            roll_outbound = np.mean(lag_outbound)
            roll_stock = np.mean(lag_stock)

            lag_features = {
                'STOCK_LAG_1': lag_stock[0],
                'STOCK_LAG_2': lag_stock[1],
                'STOCK_LAG_3': lag_stock[2],
                'INBOUND_LAG_1': lag_inbound[0],
                'INBOUND_LAG_2': lag_inbound[1],
                'INBOUND_LAG_3': lag_inbound[2],
                'OUTBOUND_LAG_1': lag_outbound[0],
                'OUTBOUND_LAG_2': lag_outbound[1],
                'OUTBOUND_LAG_3': lag_outbound[2],
                'INBOUND_ROLL_MEAN_3': roll_inbound,
                'OUTBOUND_ROLL_MEAN_3': roll_outbound,
                'STOCK_ROLL_MEAN_3': roll_stock,
                'MONTH_NUM': month_num
            }

            X_pred = pd.DataFrame([lag_features])[feature_cols]
            pred_stock = model.predict(X_pred)[0]

            forecast_results.append({
                'PLANT_NAME': plant,
                'MONTH': pred_month,
                'Predicted_Stock': pred_stock
            })

            lag_stock = [pred_stock] + lag_stock[:-1]

    return pd.DataFrame(forecast_results)

scenario_normal = run_scenario_combined(df_combined.copy(), model, feature_cols, months_ahead=6)
scenario_delay_outbound = run_scenario_combined(df_combined.copy(), model, feature_cols, months_ahead=6,
                                                outbound_delay=1)
scenario_storm = run_scenario_combined(df_combined.copy(), model, feature_cols, months_ahead=6, shocks=[
    {'start_month': 6, 'end_month': 8, 'inbound_factor': 0.3, 'outbound_factor': 0.4}
])

df_all = scenario_normal.rename(columns={'Predicted_Stock': 'Normal'})
df_all = df_all.merge(
    scenario_delay_outbound.rename(columns={'Predicted_Stock': 'Delay_Outbound'}),
    on=['PLANT_NAME', 'MONTH'],
    how='left'
)
df_all = df_all.merge(
    scenario_storm.rename(columns={'Predicted_Stock': 'Storm'}),
    on=['PLANT_NAME', 'MONTH'],
    how='left'
)

df_all.to_csv('scenario_all_combined.csv', index=False)

In [58]:
print(scenario_normal.head(6))
print(scenario_delay_outbound.head(6))
print(scenario_storm.head(6))

        PLANT_NAME      MONTH  Predicted_Stock
0  CHINA-WAREHOUSE 2025-01-01      61798.22106
1  CHINA-WAREHOUSE 2025-02-01      61782.91648
2  CHINA-WAREHOUSE 2025-03-01      59455.94013
3  CHINA-WAREHOUSE 2025-04-01      61143.63570
4  CHINA-WAREHOUSE 2025-05-01      61094.97267
5  CHINA-WAREHOUSE 2025-06-01      60896.47740
        PLANT_NAME      MONTH  Predicted_Stock
0  CHINA-WAREHOUSE 2025-01-01      61838.45590
1  CHINA-WAREHOUSE 2025-02-01      61782.91648
2  CHINA-WAREHOUSE 2025-03-01      59655.78243
3  CHINA-WAREHOUSE 2025-04-01      61143.63570
4  CHINA-WAREHOUSE 2025-05-01      61381.17528
5  CHINA-WAREHOUSE 2025-06-01      60896.47740
        PLANT_NAME      MONTH  Predicted_Stock
0  CHINA-WAREHOUSE 2025-01-01      61798.22106
1  CHINA-WAREHOUSE 2025-02-01      61782.91648
2  CHINA-WAREHOUSE 2025-03-01      59455.94013
3  CHINA-WAREHOUSE 2025-04-01      61143.63570
4  CHINA-WAREHOUSE 2025-05-01      61094.97267
5  CHINA-WAREHOUSE 2025-06-01      55487.15783
